In [ ]:
from imutils import paths
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import py360convert 

MIN_MATCH_COUNT = 8
imagePaths = sorted(list(paths.list_images("images")))
i=0
print(len(imagePaths))

for i in range(len(imagePaths)-1):
    imge1 = cv.imread(imagePaths[i])
    imge2 = cv.imread(imagePaths[i+1])
    imge1 = py360convert.e2c(imge1, face_w=1200, mode='bilinear', cube_format='dice')
    imge2 = py360convert.e2c(imge2, face_w=1200, mode='bilinear', cube_format='dice')
    imge1 = cv.cvtColor(imge1, cv.COLOR_BGR2GRAY) 
    imge2 = cv.cvtColor(imge2, cv.COLOR_BGR2GRAY) 
    imge1 = imge1[1200:3600, 0:4800]
    imge2 = imge2[1200:3600, 0:4800]
    j=0
    # Initiate SIFT detector
    sift = cv.SIFT_create()
    while j < 3601:
        img1 = imge1[0:1200, j:j+1200]
        img2 = imge2[0:1200, j:j+1200]
        print(imagePaths[i])
        print(imagePaths[i+1])

        # find the keypoints and descriptors with SIFT
        kp1, des1 = sift.detectAndCompute(img1,None)
        kp2, des2 = sift.detectAndCompute(img2,None)
        # FLANN parameters
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)
        flann = cv.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des1,des2,k=2)
        pts1 = []
        pts2 = []
        # ratio test as per Lowe's paper
        for i,(m,n) in enumerate(matches):
            if m.distance < 0.8*n.distance:
                pts2.append(kp2[m.trainIdx].pt)
                pts1.append(kp1[m.queryIdx].pt)
        pts1 = np.int32(pts1)
        pts2 = np.int32(pts2)
        F, mask = cv.findFundamentalMat(pts1,pts2,cv.FM_LMEDS)
        # We select only inlier points
        pts1 = pts1[mask.ravel()==1]
        pts2 = pts2[mask.ravel()==1]
        def drawlines(img1,img2,lines,pts1,pts2):
            ''' img1 - image on which we draw the epilines for the points in img2
                lines - corresponding epilines '''
            r,c = img1.shape
            img1 = cv.cvtColor(img1,cv.COLOR_GRAY2BGR)
            img2 = cv.cvtColor(img2,cv.COLOR_GRAY2BGR)
            for r,pt1,pt2 in zip(lines,pts1,pts2):
                color = tuple(np.random.randint(0,255,3).tolist())
                x0,y0 = map(int, [0, -r[2]/r[1] ])
                x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
                img1 = cv.line(img1, (x0,y0), (x1,y1), color,1)
                img1 = cv.circle(img1,tuple(pt1),5,color,-1)
                img2 = cv.circle(img2,tuple(pt2),5,color,-1)
            return img1,img2     
        # Find epilines corresponding to points in right image (second image) and
        # drawing its lines on left image
        lines1 = cv.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
        lines1 = lines1.reshape(-1,3)
        img5,img6 = drawlines(img1,img2,lines1,pts1,pts2)
        # Find epilines corresponding to points in left image (first image) and
        # drawing its lines on right image
        lines2 = cv.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
        lines2 = lines2.reshape(-1,3)
        img3,img4 = drawlines(img2,img1,lines2,pts2,pts1)
        plt.subplot(121),plt.imshow(img5)
        plt.subplot(122),plt.imshow(img3)
        plt.show()  
    
        j=j+1200